In [4]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import os
import pandas as pd
import numpy as np
from predict import preprocess, train_test_split, df_yielder, date_to_int
from sklearn.preprocessing import MinMaxScaler

TRAIN_RATIO = 0.2
TIMESTEP = 12


In [2]:
def create_model(input_shape):
    
    model = Sequential()
    model.add(LSTM(units=50, input_shape=input_shape))
    model.add(Dense(units=1))
    model.compile(optimizer='adam',loss='mean_squared_error')
    
    return model

In [5]:
df = pd.read_csv('data/yf_attributes/AAPL.csv')
df['Date'] = [date_to_int(date) for date in df['Date']]

scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))

scaled_X = scaler_X.fit_transform(df)
scaled_y = scaler_y.fit_transform(df[['Close']])


In [6]:
def create_sequences(data, timestep):
    X, y = [], []
    for i in range(len(data) - timestep):
        X.append(data[i:i + timestep]) #Using all data between timesteps
        y.append(data[i + timestep, 4]) #Using next timesteps close data as target
    
    return np.array(X), np.array(y)

In [23]:

X, y = create_sequences(scaled_X, TIMESTEP)

split_point = int(TRAIN_RATIO * len(X))
X_train, X_test = X[:split_point], X[split_point:]
y_train, y_test = y[:split_point], y[split_point:]

#Shape of X will be (data_length, timesteps, num_features)


In [8]:
model = create_model((X.shape[1], X.shape[2]))
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)



Epoch 1/50

3/3 [==============================] - 2s 12ms/step - loss: 6.1909e-04
Epoch 2/50
3/3 [==============================] - 0s 10ms/step - loss: 2.7482e-04
Epoch 3/50
3/3 [==============================] - 0s 9ms/step - loss: 2.4018e-04
Epoch 4/50
3/3 [==============================] - 0s 9ms/step - loss: 4.3891e-05
Epoch 5/50
3/3 [==============================] - 0s 8ms/step - loss: 1.0211e-04
Epoch 6/50
3/3 [==============================] - 0s 9ms/step - loss: 7.2258e-05
Epoch 7/50
3/3 [==============================] - 0s 9ms/step - loss: 9.9467e-06
Epoch 8/50
3/3 [==============================] - 0s 8ms/step - loss: 3.1364e-05
Epoch 9/50
3/3 [==============================] - 0s 9ms/step - loss: 2.9513e-05
Epoch 10/50
3/3 [==============================] - 0s 8ms/step - loss: 4.1417e-06
Epoch 11/50
3/3 [==============================] - 0s 8ms/step - loss: 1.2152e-05
Epoch 12/50
3/3 [==============================] - 0s 9ms/step - loss: 1.3406e-05
Epoch 13/50
3/3 [===

In [9]:
loss = model.evaluate(X_test, y_test)
print("Test Loss:", loss)

12/12 [==============================] - 0s 4ms/step - loss: 0.0927
Test Loss: 0.09274402260780334


In [24]:

last_sequence = X_test[-1].reshape(1, X_test[-1].shape[0], X_test[-1].shape[1])

next_close = model.predict(last_sequence)

descaled_next_close = scaler_y.inverse_transform(next_close)




1/1 [==============================] - 0s 23ms/step
[[-0.03367486]]
[[-6.5462074]]
